# House Price Prediction with Linear Regression


In [ ]:
!pip install numpy pandas matplotlib seaborn plotly opendatasets --quiet

## Exploring the Data



In [ ]:
dataset_url = 'https://github.com/JovianML/opendatasets/raw/master/data/house-prices-advanced-regression-techniques.zip'

In [ ]:
from urllib.request import urlretrieve

In [ ]:
urlretrieve(dataset_url, 'house-prices.zip')

In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile('house-prices.zip') as f:
    f.extractall(path='house-prices')

In [ ]:
import os

In [ ]:
data_dir = 'house-prices'

In [ ]:
os.listdir(data_dir)

In [ ]:
import pandas as pd
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

In [ ]:
train_csv_path = data_dir + '/train.csv'
train_csv_path

In [ ]:
prices_df = pd.read_csv(train_csv_path)

In [ ]:
prices_df

In [ ]:
prices_df.info()

In [ ]:
n_rows = prices_df.shape[0]
n_rows

In [ ]:
n_cols = prices_df.shape[1]
n_cols

In [ ]:
print('The dataset contains {} rows and {} columns.'.format(n_rows, n_cols))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
sns.distplot(prices_df['SalePrice']);


In [ ]:
fig = px.scatter(prices_df, x ='YearBuilt', y='SalePrice', title='YearBuilt vs SalePrice' )
fig.update_traces(marker_size=6)
fig.show()

In [ ]:
fig = px.scatter(prices_df, x ='LotArea', y='SalePrice', title='LotArea vs SalePrice' )
fig.update_traces(marker_size=6)
fig.show()

In [ ]:
fig, ax = plt.subplots(figsize=(30,30))
sns.heatmap(prices_df.corr(), cmap='Reds', annot=True, ax=ax)

plt.title('Correlation Matrix');
plt.show()

In [ ]:
prices_df.hist(figsize=(30,30), bins=35)

In [ ]:
prices_df

In [ ]:
# Identify the input columns (a list of column names)
input_cols = prices_df.columns[1:-1].tolist()
input_cols

In [ ]:
# Identify the name of the target column (a single string, not a list)
target_col = prices_df.columns[-1]
target_col

In [ ]:
print(list(input_cols))

In [ ]:
len(input_cols)

In [ ]:
print(target_col)

In [ ]:
inputs_df = prices_df[input_cols].copy()

In [ ]:
targets = prices_df[target_col]

In [ ]:
inputs_df

In [ ]:
targets

###  Numeric and Categorical Data



In [ ]:
prices_df.info()

In [ ]:
import numpy as np

In [ ]:
numeric_cols = inputs_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [ ]:
categorical_cols = inputs_df.select_dtypes(include='object').columns.tolist()

In [ ]:
print(list(numeric_cols))

In [ ]:
print(list(categorical_cols))

In [ ]:
print(len(numeric_cols))
print(len(categorical_cols))

### Impute Numerical Data



In [ ]:
missing_counts = inputs_df[numeric_cols].isna().sum().sort_values(ascending=False)
missing_counts[missing_counts > 0]

In [ ]:
#fig = px.scatter(prices_df, x ='MasVnrArea', y='SalePrice' )
#fig.update_traces(marker_size=6)
#fig.show()

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# 1. Create the imputer
imputer = SimpleImputer(strategy='mean')

In [ ]:
# 2. Fit the imputer to the numeric colums
imputer.fit(inputs_df[numeric_cols])

In [ ]:
list(imputer.statistics_)

In [ ]:
# 3. Transform and replace the numeric columns
inputs_df[numeric_cols] = imputer.transform(inputs_df[numeric_cols])

In [ ]:
missing_counts = inputs_df[numeric_cols].isna().sum().sort_values(ascending=False)
missing_counts[missing_counts > 0] # should be an empty list

### Scale Numerical Values



In [ ]:
inputs_df[numeric_cols].describe().loc[['min', 'max']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Create the scaler
scaler = MinMaxScaler()

In [ ]:
# Fit the scaler to the numeric columns
scaler.fit(inputs_df[numeric_cols])

In [ ]:
# Transform and replace the numeric columns
inputs_df[numeric_cols] = scaler.transform(inputs_df[numeric_cols])

In [ ]:
inputs_df[numeric_cols].describe().loc[['min', 'max']]

### Encode Categorical Columns



In [ ]:
inputs_df[categorical_cols].nunique().sort_values(ascending=False)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# 1. Create the encoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [ ]:
# 2. Fit the encoder to the categorical colums
encoder.fit(inputs_df[categorical_cols])

In [ ]:
# 3. Generate column names for each category
encoded_cols = list(encoder.get_feature_names(categorical_cols))
len(encoded_cols)

In [ ]:
# 4. Transform and add new one-hot category columns
inputs_df[encoded_cols] = encoder.transform(inputs_df[categorical_cols])

In [ ]:
encoder.categories_

In [ ]:
inputs_df

### Training and Validation Set



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_inputs, val_inputs, train_targets, val_targets = train_test_split(inputs_df[numeric_cols + encoded_cols], 
                                                                        targets, 
                                                                        test_size=0.25, 
                                                                        random_state=42)

In [ ]:
train_inputs

In [ ]:
train_targets

In [ ]:
val_inputs

In [ ]:
val_targets

## Train a Linear Regression Model



In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
# Create the model
model = Ridge()

In [ ]:
# Fit the model using inputs and targets
model.fit(train_inputs, train_targets)

## Predictions 


In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
train_preds = model.predict(train_inputs)

In [ ]:
train_preds

In [ ]:
train_rmse = mean_squared_error(train_targets, train_preds, squared=False)

In [ ]:
print('The RMSE loss for the training set is $ {}.'.format(train_rmse))

In [ ]:
train_targets.std()

In [ ]:
val_preds = model.predict(val_inputs)

In [ ]:
val_preds

In [ ]:
val_rmse = mean_squared_error(val_targets, val_preds, squared=False)

In [ ]:
print('The RMSE loss for the validation set is $ {}.'.format(val_rmse))

In [ ]:
val_targets.std()

### Feature Importance



In [ ]:
weights = model.coef_

In [ ]:
weights_df = pd.DataFrame({
    'columns': train_inputs.columns,
    'weight': weights
}).sort_values('weight', ascending=False)

In [ ]:
weights_df

### Making Predictions



In [ ]:
def predict_input(single_input):
    input_df = pd.DataFrame([single_input])
    input_df[numeric_cols] = imputer.transform(input_df[numeric_cols])
    input_df[numeric_cols] = scaler.transform(input_df[numeric_cols])
    input_df[encoded_cols] = encoder.transform(input_df[categorical_cols].values)
    X_input = input_df[numeric_cols + encoded_cols]
    return model.predict(X_input)[0]

In [ ]:
sample_input = { 'MSSubClass': 20, 'MSZoning': 'RL', 'LotFrontage': 77.0, 'LotArea': 9320,
 'Street': 'Pave', 'Alley': None, 'LotShape': 'IR1', 'LandContour': 'Lvl', 'Utilities': 'AllPub',
 'LotConfig': 'Inside', 'LandSlope': 'Gtl', 'Neighborhood': 'NAmes', 'Condition1': 'Norm', 'Condition2': 'Norm',
 'BldgType': '1Fam', 'HouseStyle': '1Story', 'OverallQual': 4, 'OverallCond': 5, 'YearBuilt': 1959,
 'YearRemodAdd': 1959, 'RoofStyle': 'Gable', 'RoofMatl': 'CompShg', 'Exterior1st': 'Plywood',
 'Exterior2nd': 'Plywood', 'MasVnrType': 'None','MasVnrArea': 0.0,'ExterQual': 'TA','ExterCond': 'TA',
 'Foundation': 'CBlock','BsmtQual': 'TA','BsmtCond': 'TA','BsmtExposure': 'No','BsmtFinType1': 'ALQ',
 'BsmtFinSF1': 569,'BsmtFinType2': 'Unf','BsmtFinSF2': 0,'BsmtUnfSF': 381,
 'TotalBsmtSF': 950,'Heating': 'GasA','HeatingQC': 'Fa','CentralAir': 'Y','Electrical': 'SBrkr', '1stFlrSF': 1225,
 '2ndFlrSF': 0, 'LowQualFinSF': 0, 'GrLivArea': 1225, 'BsmtFullBath': 1, 'BsmtHalfBath': 0, 'FullBath': 1,
 'HalfBath': 1, 'BedroomAbvGr': 3, 'KitchenAbvGr': 1,'KitchenQual': 'TA','TotRmsAbvGrd': 6,'Functional': 'Typ',
 'Fireplaces': 0,'FireplaceQu': np.nan,'GarageType': np.nan,'GarageYrBlt': np.nan,'GarageFinish': np.nan,'GarageCars': 0,
 'GarageArea': 0,'GarageQual': np.nan,'GarageCond': np.nan,'PavedDrive': 'Y', 'WoodDeckSF': 352, 'OpenPorchSF': 0,
 'EnclosedPorch': 0,'3SsnPorch': 0, 'ScreenPorch': 0, 'PoolArea': 0, 'PoolQC':'Ex' , 'Fence': np.nan, 'MiscFeature': 'Shed',
 'MiscVal': 400, 'MoSold': 1, 'YrSold': 2010, 'SaleType': 'WD', 'SaleCondition': 'Normal'}

In [ ]:
prices_df.PoolQC.value_counts()

In [ ]:
predicted_price = predict_input(sample_input)

In [ ]:
print('The predicted sale price of the house is ${}'.format(predicted_price))

### Saving the model



In [ ]:
import joblib

In [ ]:
house_price_predictor = {
    'model': model,
    'imputer': imputer,
    'scaler': scaler,
    'encoder': encoder,
    'input_cols': input_cols,
    'target_col': target_col,
    'numeric_cols': numeric_cols,
    'categorical_cols': categorical_cols,
    'encoded_cols': encoded_cols
}

In [ ]:
joblib.dump(house_price_predictor, 'house_price_predictor.joblib')